# 5-bus Market simulation with [PowerSimulations.jl](https://github.com/NREL-SIIP/PowerSimulations.jl)

**Originally Contributed by**: Clayton Barrows

## Introduction

PowerSimulations.jl supports simulations that consist of sequential optimization problems
where results from previous problems inform subsequent problems in a variety of ways. This
example demonstrates some of these capabilities to represent electricity market clearing.

## Dependencies and Data
First, let's create `System`s to represent the Day-Ahead and Real-Time market clearing
process with hourly, and 5-minute time series data, respectively.

### Modeling Packages

In [1]:
using PowerSystems
using PowerSimulations
using PowerSystemCaseBuilder

### Data management packages

In [2]:
using Dates
using DataFrames

### Optimization packages

In [3]:
using Cbc # mip solver
solver = optimizer_with_attributes(Cbc.Optimizer, "logLevel" => 1, "ratioGap" => 0.5)
using Ipopt # solver that supports duals
ipopt_solver = optimizer_with_attributes(Ipopt.Optimizer, "print_level" => 0)

MathOptInterface.OptimizerWithAttributes(Ipopt.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute, Any}[MathOptInterface.RawParameter("print_level") => 0])

### 5-bus Data
The five bus system data here includes hourly day-ahead data, 5-minute real-time market
data, and 6-second actual data. We'll only use the hourly and 5-minute data for the
example simulations below, but the 6-second data is included for future development.

In [4]:
sys_DA = build_system(SIIPExampleSystems, "5_bus_matpower_DA")
sys_RT = build_system(SIIPExampleSystems, "5_bus_matpower_RT")

[ Info: Loaded time series from storage file existing=5_bus_matpower_DA_time_series_storage.h5 new=/var/folders/27/2jr8c7gn4j72fvrg4qt81zrw8w_711/T/jl_x0ARWO compression=CompressionSettings(false, CompressionTypes.DEFLATE = 1, 3, true)
┌ Warning: Rate 1000.0 MW for bus1-bus5-i_3 is larger than the max expected in the range of (min = 134.0, max = 145.0).
└ @ PowerSystems ~/.julia/packages/PowerSystems/61h6O/src/utils/IO/branchdata_checks.jl:148
┌ Warning: Rate 200.0 MW for bus1-bus4-i_2 is larger than the max expected in the range of (min = 134.0, max = 145.0).
└ @ PowerSystems ~/.julia/packages/PowerSystems/61h6O/src/utils/IO/branchdata_checks.jl:148
┌ Warning: Rate 200.0 MW for bus1-bus2-i_1 is larger than the max expected in the range of (min = 134.0, max = 145.0).
└ @ PowerSystems ~/.julia/packages/PowerSystems/61h6O/src/utils/IO/branchdata_checks.jl:148
┌ Warning: Rate 200.0 MW for bus4-bus5-i_7 is larger than the max expected in the range of (min = 134.0, max = 145.0).
└ @ PowerSy

,ConcreteType,SuperTypes,Count
,String,String,Int64
1,Arc,Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,6
2,Area,AggregationTopology <: Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,1
3,Bus,Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,5
4,Line,ACBranch <: Branch <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,5
5,LoadZone,AggregationTopology <: Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,1
6,PhaseShiftingTransformer,ACBranch <: Branch <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,2
7,PowerLoad,StaticLoad <: ElectricLoad <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,3
8,RenewableDispatch,RenewableGen <: Generator <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,2
9,ThermalStandard,ThermalGen <: Generator <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,5


## `OperationsProblemTemplate`s

In [5]:
template_uc = template_unit_commitment()
template_ed = template_economic_dispatch()


Operations Problem Specification
Transmission: CopperPlatePowerModel
Devices Models: 

	Type: ThermalStandard
 	Formulation: ThermalDispatch

	Type: HydroDispatch
 	Formulation: HydroDispatchRunOfRiver

	Type: PowerLoad
 	Formulation: StaticPowerLoad

	Type: RenewableFix
 	Formulation: FixedOutput

	Type: RenewableDispatch
 	Formulation: RenewableFullDispatch

	Type: HydroEnergyReservoir
 	Formulation: HydroDispatchRunOfRiver

	Type: InterruptibleLoad
 	Formulation: InterruptiblePowerLoad

Branches Models: 

	Type: Line
 	Formulation: StaticBranch

	Type: TapTransformer
 	Formulation: StaticBranch

	Type: Transformer2W
 	Formulation: StaticBranch

	Type: HVDCLine
 	Formulation: HVDCDispatch

Services Models:

	Type: VariableReserve{ReserveDown}
 	Formulation: RangeReserve

	Type: VariableReserve{ReserveUp}
 	Formulation: RangeReserve



### Define the Simulation Sequence

In [6]:
problems = SimulationProblems(
    UC = OperationsProblem(
        template_uc,
        sys_DA,
        optimizer = solver,
        balance_slack_variables = true,
    ),
    ED = OperationsProblem(
        template_ed,
        sys_RT,
        optimizer = ipopt_solver,
        constraint_duals = [:CopperPlateBalance],
    ),
)

feedforward_chronologies = Dict(("UC" => "ED") => Synchronize(periods = 24))

feedforward = Dict(
    ("ED", :devices, :ThermalStandard) => SemiContinuousFF(
        binary_source_problem = ON,
        affected_variables = [ACTIVE_POWER],
    ),
)

#cache = Dict("UC" => [TimeStatusChange(ThermalStandard, PSI.ON)])
intervals = Dict("UC" => (Hour(24), Consecutive()), "ED" => (Hour(1), Consecutive()))

DA_RT_sequence = SimulationSequence(
    problems = problems,
    intervals = intervals,
    ini_cond_chronology = InterProblemChronology(),
    feedforward_chronologies = feedforward_chronologies,
    feedforward = feedforward,
)

Feed Forward Chronology
-----------------------

ED: SemiContinuousFF -> ThermalStandard

                     UC--┐ from : On
                         |
┌----┬----┬----┬----┬----┼----┬----┬----┬----┬----┬----┐
|    |    |    |    |    |    |    |    |    |    |    |
|    |    |    |    |    |    |    |    |    |    |    |
└─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED ... (x24) to : ["P"]

Initial Condition Chronology
----------------------------

1
|
|
2 --> 2 ... (x24)   


## `Simulation`

In [7]:
file_path = mkpath(joinpath(".", "5-bus-simulation"))
sim = Simulation(
    name = "5bus-test",
    steps = 1,
    problems = problems,
    sequence = DA_RT_sequence,
    simulation_folder = file_path,
)

Simulation()


### Build simulation

In [8]:
build!(sim)

┌ Info: 
│  ──────────────────────────────────────────────────────────────────────────────
│                                        Time                   Allocations      
│                                ──────────────────────   ───────────────────────
│        Tot / % measured:            1.19s / 100%             141MiB / 100%     
│ 
│  Section               ncalls     time   %tot     avg     alloc   %tot      avg
│  ──────────────────────────────────────────────────────────────────────────────
│  Build Simulation           1    1.19s   100%   1.19s    141MiB  100%    141MiB
│    Build Problems           1    1.18s   100%   1.18s    140MiB  100%    140MiB
│      Problem UC             1    760ms  63.9%   760ms   95.6MiB  68.0%  95.6MiB
│        ThermalStandard      1   32.3ms  2.72%  32.3ms   5.78MiB  4.11%  5.78MiB
│        Services             1   25.2ms  2.12%  25.2ms   4.24MiB  3.02%  4.24MiB
│        RenewableDis...      1   12.4ms  1.04%  12.4ms   1.30MiB  0.92%  1.30MiB
│   

BuildStatus.BUILT = 0

### Execute simulation
```julia

In [9]:
execute!(sim, enable_progress_bar = false)

RunStatus.SUCCESSFUL = 0

```

In [10]:
# Results

First we can load the result metadata
```julia
results = SimulationResults(sim);
uc_results = get_problem_results(results, "UC")
ed_results = get_problem_results(results, "ED");
```

Then we can read and examine the results of interest
```julia
prices = read_dual(ed_results, :CopperPlateBalance)
```

or if we want to look at the realized values
```julia
read_realized_duals(ed_results)[:CopperPlateBalance]
```

*note that in this simulation the prices are all equal to the balance slack
penalty value of $100000/MWh because there is unserved energy in the result*

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*